Bryan Flores <br>
ADS 509 <br>
Assignment 4.1 <br>
February 5, 2024

## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [1]:
import sqlite3
from sqlite3 import Error
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
import re
import pandas as pd
import string

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
def get_engine(database):
    """
    Create database connection.
    :param database: database file
    :return convention_cur: connection object
    """
    conn = None
    try:
        conn = sqlite3.connect(database)
        curr = conn.cursor()
    except Error as e:
        print(e)

    return curr

In [3]:
convention_cur = get_engine("2020_Conventions.db")

In [4]:
# View db objects
query = "SELECT name FROM sqlite_master WHERE type = 'table'"
convention_cur.execute(query).fetchall()[0][0]

'conventions'

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [5]:
# Stopwords
sw = set(nltk.corpus.stopwords.words("english"))

def tokenize(text) : 
    """ Tokenize text.
    :param text: str, 
    :return: tokenize strings for analysis
    """
    input_text = re.sub(r'[^\w\s]', '', text)
    
    # remove punctuation, lower, and tokenize
    translator = str.maketrans('', '', string.punctuation)
    clean_string = input_text.translate(translator)
    
    tokens_nostop = ' '.join([w for w in clean_string.lower().split() if w not in sw])
    
    return tokens_nostop

In [6]:
convention_data = []

# fill this list up with items that are themselves lists. 
# The first element in the sublist should be the cleaned and tokenized
#   text in a single string. 
# The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            select 
                                party,
                                text
                            from conventions
                            ''').fetchall()

party = [row[0] for row in query_results]
text = [tokenize(row[1]) for row in query_results]
# Append lists to convention data
convention_data.append(text)
convention_data.append(party)

# print(convention_data)

Let's look at some random entries and see if they look right. 

In [7]:
# random.choices(convention_data, k=10)

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [8]:
word_cutoff = 5

tokens = [w for t in convention_data[0] for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2378 as features in the model.


In [9]:
def conv_features(text, fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
    """
    # Check if text is a list, then tokenize continuous string
    if type(text) is list:
        tokens = [w for t in convention_data[0] for w in t.split()]
    else:
        tokens = text.split()

    # Find tokens matching tokens and feature words
    ret_dict = dict()
    for token in tokens:
        if token in fw:
            ret_dict.update({token: True})
        else:
            pass

    # Check for duplicate key-values
    value_occurrences = Counter(ret_dict.values())
    filtered_dict = {key: value for key, value in ret_dict.items() if value_occurrences[value] > 1}

    return filtered_dict

In [10]:
# Test function
test_string = convention_data[0][0]
test_fw = {'skip', 'skip', 'content', 'foreign'}
conv_features(text=test_string, fw=test_fw)

{'skip': True, 'content': True, 'foreign': True}

In [11]:
assert(len(feature_words) > 0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,"citizens":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [12]:
conv_df = pd.DataFrame(list(zip(text, party)), columns=['text', 'party'])
conv_df.head()

,text,party
0,skip content company careers press freelancers...,Democratic
1,im calling full session 48th quadrennial natio...,Democratic
2,every four years come together reaffirm democr...,Democratic
3,fight perfect union fighting soul country live...,Democratic
4,must come together defeat donald trump elect j...,Democratic


In [13]:
conv_df['featuresets'] = [(conv_features(t, feature_words), p) for (t, p) in zip(conv_df['text'], conv_df['party'])]
conv_df.head()

,text,party,featuresets
0,skip content company careers press freelancers...,Democratic,"({'skip': True, 'content': True, 'company': Tr..."
1,im calling full session 48th quadrennial natio...,Democratic,"({'im': True, 'calling': True, 'full': True, '..."
2,every four years come together reaffirm democr...,Democratic,"({'every': True, 'four': True, 'years': True, ..."
3,fight perfect union fighting soul country live...,Democratic,"({'fight': True, 'perfect': True, 'union': Tru..."
4,must come together defeat donald trump elect j...,Democratic,"({'must': True, 'come': True, 'together': True..."


In [14]:
random.seed(20220507)
random.shuffle(conv_df['featuresets'])

test_size = 500

In [15]:
test_set, train_set = conv_df['featuresets'][:test_size], conv_df['featuresets'][test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [16]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

All except 2 of the top 25 words are classified as Republican. Moreover, there is a general theme of military-related terms and patriotism. From a glance, Republicans speeches stick to a few specific topics, while Democratic speeches are more broad. 

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [17]:
cong_cur = get_engine("congressional_data.db")

In [18]:
query = "SELECT name FROM sqlite_master WHERE type = 'table'"
cong_cur.execute(query).fetchall()

[('websites',), ('candidate_data',), ('tweets',)]

In [19]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [23]:
tweet_data = [[tokenize(row[2].decode('utf-8')), row[1]] for row in results]

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

# party = [row[1] for row in results]
# text = [tokenize(row[2].decode('utf-8')) for row in results]
# # Append lists to convention data
# tweet_data.append(party)
# tweet_data.append(text)

print(tweet_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [24]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data, k=10)

In [25]:
for tweet, party in tweet_data_sample:
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: lets make even greater kag httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1hr cavs tie se

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [26]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [28]:
pd.DataFrame(results)

,Republican,Democratic
Republican,3575,4635
Democratic,703,1089


### Reflections

The results more easily identify Republican text over Democratic text, which we can see in the 4635 incorrectly predicted Democratic results. 